# Functional Programming and Data Engineering

## 1. Functional Programming: Pure functions

Mathematics functions are what are known as pure functions. Given the same inputs, they always give the same output, and have no side effects. For example:

In [ ]:
f = lambda x, y: x*x + y*y
f(2,3)

13

If you come back a few days later, you will get the same answer:

In [ ]:
f(2,3)

13

But in programming languages you can write "impure" functions because of how the scope of variables act:

In [ ]:
jdict = {}
jdict['out'] = 0
def impure_f(i):
    jdict['out'] = jdict['out'] + 1
    return 3

In [ ]:
impure_f(5)

3

In [ ]:
jdict

{'out': 1}

In [ ]:
impure_f(5)

3

In [ ]:
jdict

{'out': 2}

Each call to this function, while even returning the same thing, has a side-effect. This can easily extend to return values with the same input

In [ ]:
jdict = {}
jdict['out'] = 0
def impure_f2(i):
    jdict['out'] = jdict['out'] + 1
    return jdict['out']

In [ ]:
jdict

{'out': 0}

In [ ]:
impure_f2(5)

1

In [ ]:
jdict

{'out': 1}

In [ ]:
impure_f2(5)

2

In [ ]:
jdict

{'out': 2}

The reason for this is that the function has used variables in our program in a way that includes state. As we shall learn soon in object oriented programming, state is an important thing to consider in programs, How else are we to reason about the passage of time?

But state in functions, even included inadvertently, can lead to problems. Without seeing the full flow of the program above, you do not know ahead of time, what is going to happen on any call of `impure_f` or `impure_f2`. This makes it very hard to reason about your program. 

So, in general, when you write a function like this, you want the return value to be the same given the inputs, and for there to be no side effects. Practically this means: output a new list or new dictionary: don't change a variable that already exists.

Doing this allows a whole lot of guarantees that help make programs simple: a pure function can be run on another machine or even later in time, as it is guaranteed to give the same result. This is the genesis of the [famous map-reduce paper from google](https://static.googleusercontent.com/media/research.google.com/en//archive/mapreduce-osdi04.pdf)  and the follow-on hadoop software which have been critical for working with data. But these do require us to work with state and time differently, as we shall later see.

## 2. Functional Programming: Functions that manipulate functions: composition of these functions

One way to think about functional programming is in the quality of glue it provides to make modular programming easier.

As John Hughes writes in "Why Functional Programming Matters":

>When writing a modular program to solve a problem, one first divides the problem into subproblems, then solves the subproblems, and finally combines the solutions. The ways in which one can divide up the original problem depend directly on the ways in which one can glue solutions together. Therefore, to increase one’s ability to modularize a problem conceptually, one must provide new kinds of glue in the programming language.

The beauty of being able to take functions as arguments, and to return them is that it makes possible better glue. We shall talk about one such glue here:

Lets motivate this by thinking about how to find the sum of all elements in a list:

In [ ]:
def fsum(alist):
    if len(alist)==0: # empty list
        return 0
    else:
        return alist[0] + fsum(alist[1:])

In [ ]:
fsum([])

0

In [ ]:
fsum([1])

1

In [ ]:
fsum([1,2,3])

6

Notice that this is a recursive function that calls itself on the remaining part of the list. And this will continue until nothing is left. Now you would now right a sum in python like this, because it would be slow and take up a lot of memory, but let us use it as a conceptual device.

Now what if you wanted to do this for multiplication? You could say the product of an empty list is 1, and then you just multiply the items in the list recursively.

In [ ]:
def fprod(alist):
    if len(alist)==0: # empty list
        return 1
    else:
        return alist[0] * fprod(alist[1:])

In [ ]:
fprod([]), fprod([1]), fprod([1,2]), fprod([1,2,3])

(1, 1, 2, 6)

Do the forms of these two functions look similar to you? Since computer science is all about building towers of abstractions, we ought to be looking at perhaps something like this?

```python
def f_op(alist, op, startval):
    if len(alist)==0: # empty list
        return startval
    else:
        return alist[0] op f_op(alist[1:])
```

In [ ]:
import operator
def f_op(alist, op, startval):
    if len(alist)==0: # empty list
        return startval
    else:
        return op(alist[0], f_op(alist[1:], op, startval))
f_op([1,2,3], operator.add, 0)

6

In [ ]:
f_op([1,2,3], operator.mul, 1)

6

Here we have expressed addition and multiplication as functions (which is what they are under the hood in python, the usual symbols are merely syntactical sugar).

Critically, we have been able to express both the sum and the product as the higher order result of passing the plus and product "operators" to a generic `f_op` function. This sort of function is also called `foldr` in the literature. And its not just for numbers...

In [ ]:
def cons(a,b):
    return [a] + b
f_op([1,2,3], cons, [])

[1, 2, 3]

This constructs back the list as a copy. What if you want to append?

In [ ]:
f_op([1,2,3], cons, [9, 10])

[1, 2, 3, 9, 10]

What if you wanted to double any element in the list?

In [ ]:
def double_and_cons(a, b):
    return [2*a] + b
f_op([1,2,3], double_and_cons, [])

[2, 4, 6]

On but what if we wrote this slightly differently:

In [ ]:
double = lambda j: 2*j
def f_and_cons_composer(f):
    def f_and_cons(a, b):
        return [f(a)] + b
    return f_and_cons
double_and_cons2 = f_and_cons_composer(double)

In [ ]:
f_op([1,2,3], double_and_cons2, [])

[2, 4, 6]

In [ ]:
def f_map(f, alist):
    f = f_and_cons_composer(f)
    return f_op(alist, f, [])

In [ ]:
f_map(double, [1,2,3])

[2, 4, 6]

So what is the overarching story here? By modularizing summation as a combination of a “higher-order function” and an operator, we have arrived at a part (f_op) that can be used to write many other functions on lists with no more programming effort. And then we used it and a function that returns a function, which as can be seen implements a composition of functions, to arrive at a simple `map` which takes every element and does something to it, implemented in terms of `f_op`.

So to summarize, a higher-order-function: a function that calls a function and can return a function combines with a specializing function like `double` to make writing more complex programs easy. And because these functions are pure, we can move around their execution, to different points in time, but also to different machines, and if something goes wrong, we just have to do the function again.

To generalize this to other data structures like trees, we'll have to write `foldtree` and `maptree`, general ways of visiting that data structure. And combine with specializing functions in the same way. These constructs lead to easy parallelization, and thus we get map-reduce from google and its successor Hadoop.

## Composition of entire functional programs and lazy evaluation

Newton's algorithm computes the square root of a number $n$ by starting from an initial approximation $a_0$ and computing better and better ones using the rule
$$
a_{i+1}=\left(a_i+n / a_i\right) / 2
$$

If the approximations converge to some limit $a$, then
$$
a=(a+n / a) / 2
$$
so
$$
\begin{aligned}
2 a & =a+n / a \\
a & =n / a \\
a * a & =n \\
a & =\sqrt{n}
\end{aligned}
$$

In fact the approximations converge rapidly to a limit. Square root programs take a tolerance (eps) and stop when two successive approximations differ by less than eps.

In [ ]:
def square_root(n, eps, a0):
    x_prev = a0
    while True:
        assert x_prev!=0
        x_next = 0.5*(x_prev + n/x_prev)
        print(x_next)
        if abs(x_next - x_prev) < eps:
            return x_next
        else:
            x_prev = x_next

In this imperative implementation the checking of the condition and the algorithm are intertwined and inseparable. The exit happens on a particular branch of the function. State is stored in `x_prev`: this is not a pure function.

In [ ]:
square_root(9, 0.0001, 1)

5.0
3.4
3.023529411764706
3.00009155413138
3.000000001396984


3.000000001396984

We can write this code in a somewhat more functional way by composing some pure functions:

In [ ]:
def good_enough_maker(eps):
    def is_it_good_enough(x_next, x_prev):
        if abs(x_next - x_prev) < eps:
            return True
        return False
    return is_it_good_enough

In [ ]:
good_enough = good_enough_maker(0.0001)

In [ ]:
def improve(n, x_prev):
    return 0.5*(x_prev + n/x_prev)

And compose a recursive routine which we can spread on a cluster ala hadoop, since these are all pure functions.

In [ ]:
def sqrt_iter(guess, n):
    x_next = improve(n, guess)
    if good_enough(x_next, x_prev):
        return x_next
    else:
        return sqrt_iter(x_next, n)
    

What if we thought of this program instead as a composition of two programs. One would run the iterations of the square-root routine while the other one would check the condition. If one made the inner function the iteration one and the outer the check one, one could consider a program composition:

`check(iter())`. The interesting thing about this idea is that one could run the outer function first, and only if necessary, run the inner one, helping save compute, even if the inner program runs infinitely, as a recursive program might. This idea is called **lazy evaluation**, or evaluate program `iter()` only if you need to.

How might one do this? We can start by redefining good_enough thus:

In [ ]:
def within_maker(eps, n):
    def is_it_within(x_next, x_prev):
        if abs(x_next - x_prev) < eps:
            return x_next
        return is_it_within(improve(n, x_next), x_next)
    return is_it_within

In [ ]:
within = within_maker(0.0001, 9)

In [ ]:
within(improve(9, 1),1)

3.000000001396984

This may seem like a very small change, but it has reversed the order of execution. `within` runs first, and improve only runs if needed. And these are pure functions.

One can improve the ergonomics of these functions by returning data structures..., and in the process removing any call to improve from inside within, so that they can be completely decoupled and within can be used for any iterative process). But for that we'd need to introduce `yield` in python and get too advanced: because what we want to do is pause the inner function, returning control to the outside, which only returns control to the inside if needed.

And this leads to one of the core ideas behind spark: a code optimizer determines whether inner functions need to run at all.